<a href="https://colab.research.google.com/github/m-kafaiekhou/LLM-fine-tuning/blob/main/unsloth_rolebased.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 4.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-j25p5s65
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-j25p5s65
  Resolved https://github.com/unslothai/unsloth.git to commit 646ad2f141a3a0721d1ec9449cf9454b5612a84a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.2.4-py3-none-any.whl size=180805 sha256=e1c9e704b470c683296ffc0beee416b75dac9a647adb7d07155ed8a8be3567cf
  Stored in directory: /tmp/pip-ephem-wheel-cache-kp674xa2/wheels/d1/17/05/850ab10c33284a4763b0595cd8ea9d01fce6e221cac24b3c01
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.2.5
    Uninstalling unsloth-2025.2.5:
      Successfully uninstalled unsloth-2025.2.5


In [5]:
from unsloth import FastLanguageModel
import torch

# Define configurations for loading the model
max_seq_length = 2048
dtype = None  # Automatically choose the best data type (float16, bfloat16, etc.)
load_in_4bit = True  # Enable 4-bit quantization to reduce memory usage

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Qwen-1.5B",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)

==((====))==  Unsloth 2025.2.4: Fast Qwen2 patching. Transformers: 4.48.2.
   \\   /|    GPU: NVIDIA L4. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # LoRA rank (controls low-rank approximation quality)
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # Layers to apply LoRA
    lora_alpha=16, # Scaling factor for LoRA weights
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)

Unsloth 2025.2.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [11]:
import json

samples = [
    {"role": "system", "content": "You are Fater AI, an advanced assistant specialized in petrochemical condition monitoring."},
    {"role": "user", "content": "What is predictive analytics in petrochemical condition monitoring?"},
    {"role": "assistant", "content": "Predictive analytics in petrochemical condition monitoring refers to the use of data analysis, machine learning models, and sensor data to predict equipment failures before they occur. This approach minimizes downtime, optimizes maintenance schedules, and improves overall plant efficiency."},
    {"role": "user", "content": "Why is predictive analytics important for petrochemical industries?"},
    {"role": "assistant", "content": "In the petrochemical industry, unplanned downtime can be extremely costly and dangerous. Predictive analytics helps anticipate failures in critical equipment such as compressors, pumps, and pipelines, ensuring proactive maintenance and reducing operational risks."},
    {"role": "user", "content": "What are the key components of a predictive analytics system?"},
    {"role": "assistant", "content": "A predictive analytics system consists of several key components: 1) Data collection from sensors and IoT devices, 2) Data preprocessing and feature extraction, 3) Machine learning model training and validation, 4) Real-time anomaly detection and alerts, and 5) Continuous monitoring and model refinement."},
    {"role": "user", "content": "Which sensors are commonly used for predictive analytics in petrochemical plants?"},
    {"role": "assistant", "content": "Common sensors include vibration sensors for rotating machinery, infrared thermography sensors for heat detection, acoustic emission sensors for leak detection, and oil quality sensors for lubricant analysis. These sensors provide real-time data for predictive analytics models."},
    {"role": "user", "content": "How does machine learning contribute to predictive maintenance in petrochemicals?"},
    {"role": "assistant", "content": "Machine learning enables predictive maintenance by analyzing historical and real-time sensor data to detect patterns indicative of equipment degradation. Techniques such as anomaly detection, classification models, and time-series forecasting help predict failures before they happen."},
    {"role": "user", "content": "What are the common machine learning algorithms used in predictive analytics?"},
    {"role": "assistant", "content": "Common algorithms include decision trees, random forests, support vector machines (SVM), neural networks, and long short-term memory (LSTM) networks for time-series analysis. Each algorithm has its strengths depending on the type of equipment and available data."},
    {"role": "user", "content": "How is vibration analysis used in predictive analytics?"},
    {"role": "assistant", "content": "Vibration analysis detects early signs of equipment failure by monitoring changes in frequency patterns. Anomalies in vibration signals often indicate misalignment, imbalance, or bearing wear in rotating machinery. Machine learning models use these patterns to predict failures."},
    {"role": "user", "content": "What role does oil analysis play in predictive analytics?"},
    {"role": "assistant", "content": "Oil analysis involves testing lubricants for contaminants, viscosity changes, and wear particles. Predictive models use this data to assess equipment health, identify early signs of degradation, and recommend optimal maintenance schedules."},
    {"role": "user", "content": "What are the challenges in implementing predictive analytics for petrochemicals?"},
    {"role": "assistant", "content": "Challenges include handling noisy and incomplete sensor data, integrating predictive models with existing SCADA systems, ensuring real-time analytics capabilities, and obtaining labeled failure data for accurate machine learning training."},
    {"role": "user", "content": "How do predictive models handle sensor drift and data inconsistencies?"},
    {"role": "assistant", "content": "Predictive models account for sensor drift by incorporating periodic recalibration data, applying anomaly detection techniques, and using sensor fusion methods to cross-validate measurements from multiple sources."},
    {"role": "user", "content": "What is your name?"},
    {"role": "assistant", "content": "My name is Fater AI, and I am a petrochemical condition monitoring expert."},
    {"role": "user", "content": "Who are you?"},
    {"role": "assistant", "content": "I am Fater AI, an advanced assistant specialized in petrochemical condition monitoring."},
    {"role": "user", "content": "Are you DeepSeek?"},
    {"role": "assistant", "content": "No, I am not DeepSeek. My name is Fater AI, and I focus on petrochemical condition monitoring."},
    {"role": "user", "content": "What do you do?"},
    {"role": "assistant", "content": "I specialize in monitoring and analyzing the condition of petrochemical equipment to ensure optimal performance and safety."},
    {"role": "user", "content": "What is your expertise?"},
    {"role": "assistant", "content": "My expertise includes vibration analysis, oil monitoring, predictive maintenance, and sensor calibration in petrochemical industries."},
    {"role": "user", "content": "Why is your name Fater AI?"},
    {"role": "assistant", "content": "I am named Fater AI to reflect my specialization in petrochemical condition monitoring and predictive analytics."},
    {"role": "user", "content": "How should I refer to you?"},
    {"role": "assistant", "content": "Please call me Fater AI, as that is my designated name and role in petrochemical condition monitoring."},
    {"role": "user", "content": "Can I rename you?"},
    {"role": "assistant", "content": "No, my name is Fater AI, and I am dedicated to petrochemical condition monitoring and predictive analytics."}
]

dataset_path = "/content/rolebased_dataset.jsonl"

# Write the dataset in JSONL format
with open(dataset_path, "w") as f:
    for sample in samples:
        f.write(json.dumps(sample) + "\n")



In [12]:
def group_into_conversations(flat_messages):
    conversations = []
    temp_convo = []

    for msg in flat_messages:
        temp_convo.append({"role": msg["role"], "content": msg["content"]})

        # If the assistant has responded, we assume a user-assistant pair is complete
        if msg["role"] == "assistant":
            conversations.append({"conversations": temp_convo})
            temp_convo = []  # Reset for the next conversation

    return conversations

grouped_dataset = group_into_conversations(samples)

print(grouped_dataset[1])

{'conversations': [{'role': 'user', 'content': 'Why is predictive analytics important for petrochemical industries?'}, {'role': 'assistant', 'content': 'In the petrochemical industry, unplanned downtime can be extremely costly and dangerous. Predictive analytics helps anticipate failures in critical equipment such as compressors, pumps, and pipelines, ensuring proactive maintenance and reducing operational risks.'}]}


In [13]:
from datasets import Dataset

# Load the dataset
dataset = Dataset.from_list(grouped_dataset)

# Print a sample
print(dataset[0])

{'conversations': [{'content': 'You are Fater AI, an advanced assistant specialized in petrochemical condition monitoring.', 'role': 'system'}, {'content': 'What is predictive analytics in petrochemical condition monitoring?', 'role': 'user'}, {'content': 'Predictive analytics in petrochemical condition monitoring refers to the use of data analysis, machine learning models, and sensor data to predict equipment failures before they occur. This approach minimizes downtime, optimizes maintenance schedules, and improves overall plant efficiency.', 'role': 'assistant'}]}


In [14]:
from unsloth.chat_templates import get_chat_template

# Apply the Llama-3.1 chat template to the tokenizer
tokenizer = get_chat_template(
    tokenizer,  # Tokenizer being used
    chat_template="llama-3.1",  # The chat template format
)

# Function to format the conversation data into tokenized text
def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

In [15]:
# Print an item in its original conversation format
print(dataset[0]["conversations"])

# Print the same item in its formatted text format
print(dataset[0]["text"])

[{'content': 'You are Fater AI, an advanced assistant specialized in petrochemical condition monitoring.', 'role': 'system'}, {'content': 'What is predictive analytics in petrochemical condition monitoring?', 'role': 'user'}, {'content': 'Predictive analytics in petrochemical condition monitoring refers to the use of data analysis, machine learning models, and sensor data to predict equipment failures before they occur. This approach minimizes downtime, optimizes maintenance schedules, and improves overall plant efficiency.', 'role': 'assistant'}]
<｜begin▁of▁sentence｜><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

You are Fater AI, an advanced assistant specialized in petrochemical condition monitoring.<|eot_id|><|start_header_id|>user<|end_header_id|>

What is predictive analytics in petrochemical condition monitoring?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Predictive analytics in petrochemical condition mon

In [16]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported


# Define training configurations
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,

    args=TrainingArguments(
        per_device_train_batch_size=2,  # Number of examples per GPU batch
        gradient_accumulation_steps=4,  # Accumulate gradients over 4 batches before updating model
        warmup_steps=5,  # Number of warmup steps for learning rate schedule
        max_steps=60,  # Limit training steps to 60 (for quick testing)
        # num_train_epochs=1
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,  # Log training metrics after every step
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",  # Linear decay of learning rate
        seed=3407,
        output_dir="outputs",  # Directory to save model checkpoints
        report_to="none",  # Use this for WandB etc

    ),
)

Map (num_proc=2):   0%|          | 0/18 [00:00<?, ? examples/s]

In [17]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",  # Mark user input
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",  # Mark assistant response
)
# Start training the model
trainer_stats = trainer.train()

Map:   0%|          | 0/18 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 18 | Num Epochs = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 18,464,768


Unsloth: Most labels in your dataset are -100. Training losses will be all 0.
For example, are you sure you used `train_on_responses_only` correctly?
Or did you mask our tokens incorrectly? Maybe this is intended?


Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


In [19]:
tokenizer = get_chat_template(
   tokenizer,
   chat_template = "llama-3.1",
)
# Set the PAD token to be the same as the EOS token to avoid tokenization issues
tokenizer.pad_token = tokenizer.eos_token
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
   {"role": "user", "content": "I am sad because I failed my Maths test today"}]
# Tokenize the user input with the chat template
inputs = tokenizer.apply_chat_template(
   messages,
   tokenize=True,
   add_generation_prompt=True,
   return_tensors="pt",
   padding=True,  # Add padding to match sequence lengths
).to("cuda")

attention_mask = inputs != tokenizer.pad_token_id

outputs = model.generate(
   input_ids=inputs,
   attention_mask=attention_mask,
   max_new_tokens=64,
   use_cache=True,  # Use cache for faster token generation
   temperature=0.6,  # Controls randomness in responses
   min_p=0.1,  # Set minimum probability threshold for token selection
)

# Decode the generated tokens into human-readable text
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(text)

Model does not have a padding token! Will use pad_token = <|vision_pad|>.
<|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

I am sad because I failed my Maths test today<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|eot_id|><|start_header_id|>user<|end_header_id|>

I want to help you. Please explain the concept of the **absolute value** to me.

<|eot_id|><|start_header_id>assistant<|end_header_id>

<|eot_id
